# Code zu Lanczos-Verfahren mithilfe von QR-Verfahren

In [1]:
import numpy as np
from scipy.stats import unitary_group
from scipy.linalg import block_diag
import time
import matplotlib.pyplot as plt

### Funktionen zum generiern von Matrizen

In [282]:
def gen_hess(n, compl = False):
    if compl:
        a = np.random.rand(n) + np.random.rand(n)*1j
        b = np.random.rand(n-1) + np.random.rand(n-1)*1j
        c = np.random.rand(n-1) + np.random.rand(n-1)*1j
        return np.diag(b, -1) + np.diag(a,0) + np.diag(c, 1)
    else:
        A = np.zeros((n,n))
        a = np.random.rand(n)*50
        for j in range(n-1):
            b = np.random.rand(n-j-1)*50
            A = A + np.diag(b,j+1)
        return np.diag(np.random.rand(n-1)*50, -1) + A
    

def gen_upper_tri(eig, compl = False):
    n = eig.shape[0]
    if compl:
        return np.triu(np.random.rand(n,n) + np.random.rand(n,n) * 1j, 1) + np.diag(eig)
    else:
        return np.triu(np.random.rand(n,n),1) + np.diag(eig)
    

def gen_hermite(eig, compl = False):
    n = eig.shape[0]
    B = unitary_group.rvs(n)
    return B.T.conj()@np.diag(eig)@B


def gen_rand_mat(eig, compl = False):
    n = eig.shape[0]
    if compl:
        B = np.random.rand(n,n) + np.random.rand(n,n)*compl*1j
    else:
        B = np.random.rand(n,n)
    return np.linalg.inv(B)@np.diag(eig)@B



#print(gen_hess(5))
#print(gen_upper_tri(np.array([1,2,3]), True))
#print(gen_hermite(np.array([1,2,3,4])))
#print(gen_rand_mat(np.array([1,2,7,4])))



### Verfahren implementiert

In [308]:
def QR_simple(A,tol = 1e-6):
    count = 0
    while abs(A[1,0]) > tol:
        Q,R = np.linalg.qr(A)
        A = R@Q
        count +=1
    return A, sorted(np.diag(A)) ,count



def QR_shift(A,tol=1e-10):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol:
            rho = A[i,i]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count +=1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_shift2(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = np.linalg.qr(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_hesse(A,tol=1e-12):
    n = A.shape[1]
    count = 0
    for i in range(n-1,0,-1):
        while abs(A[i,i-1]) > tol*(abs(A[i-1,i-1])+abs(A[i,i])):
            w = np.linalg.eigvals(A[i-1:i+1,i-1:i+1])
            if abs(w[0] - A[i,i]) < abs(w[1] - A[i,i]):
                rho = w[0]
            else:
                rho = w[1]
            Q,R = QR_decomp_hesse(A-rho*np.identity(n))
            A = R@Q + rho*np.identity(n)
            count += 1
        A[i,:i-n] = 0
    return A, sorted(np.diag(A)), count



def QR_decomp_hesse(A):
    n = A.shape[0]
    Q = np.eye(n,n)

    for i in range(0,n-1):
        if abs(A[i,i]) >= abs(A[i+1,i]):
            t = A[i+1,i]/abs(A[i,i])
            root = (1+abs(t)**2)**(1/2)
            c = A[i,i]/(abs(A[i,i])*root)
            s = t/root
        else:
            t = A[i,i]/abs(A[i+1,i])
            root = (1+abs(t)**2)**(1/2)
            s = A[i+1,i]/(abs(A[i+1,i])*root)
            c = t/root
            
        #M = np.array([[c.conj(), s.conj()], [-s, c]])
        #G = block_diag(np.eye(i,i), M, np.eye(n-i-2, n-i-2))
        #Q = G@Q
        for j in range(i, n):
            temp_1 = A[i,j]
            A[i,j] = c.conj()*temp_1 + s.conj()*A[i+1,j]
            A[i+1,j] = -s*temp_1 + c*A[i+1,j]
        for j in range(n):
            temp_2 = Q[i,j]
            Q[i,j] = c.conj()*temp_2 + s.conj()*Q[i+1,j]
            Q[i+1,j] = -s*temp_2 + c*Q[i+1,j]
    return Q.T.conj(), A



def lanczos(A,k = 0):
    n = A.shape[1]
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    gam = [v[0].T.conj()@A@v[0]]
    w = (A - gam[0]*np.identity(n))@v[0]
    delta = [np.linalg.norm(w)]
    i = 0
    while delta[i] > 1e-10 and i < k-1:
        v.append(w/delta[i])
        i +=1
        gam.append(v[i].T.conj()@A@v[i])
        w = (A - gam[i]*np.identity(n))@v[i] - delta[i-1]*v[i-1]
        delta.append(np.linalg.norm(w))
    T = np.diag(delta[:-1], -1) + np.diag(gam) + np.diag(delta[:-1], 1)
    return QR_hesse(T)

def arnoldi(A, dim, k = 0):
    n = dim
    if k == 0:
        k = n
        
    v0 = np.random.rand(n)
    v = [v0/np.linalg.norm(v0)]
    h = np.zeros((k,k))
    
    for j in range(k):
        w = A(v[j])
        for l in range(j+1):
            h[l][j] = w.T.conj()@v[l]
            w = w - h[l][j]*v[l]
        if j < k-1:
            h[j+1][j] = np.sqrt(w.T.conj()@w)
            if abs(h[j+1][j]) < 1e-14:
                return QR_hesse(h[:j+1,:j+1])
            else:
                v.append(w/h[j+1][j])
    return QR_hesse(h)

### Testing

In [312]:
def error(eig_1, eig_2):
    eig_1 = np.sort(eig_1)
    eig_2 = np.sort(eig_2)
    return np.linalg.norm(eig_1 - eig_2)


def runtime(f,A):
    start = time.time()
    f(A)
    end = time.time()
    return end - start

def runtime_error(f,A):
    start = time.time()
    S = f(A)
    end = time.time()
    return end - start, S

def runtime_error_lanczos(f,A,k):
    start = time.time()
    S = f(A,k)
    end = time.time()
    return end - start, S

def testing(f, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error(f,A)
        err = error(A[1], eigv)
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()

    
def testing_lanczos(f,k, n_1, n_2):
    n_array = np.array([i for i in range(n_1,n_2)])
    time_array = []
    error_array = []
    for n in n_array:
        eigv = np.sort(50*np.random.rand(n)-25)
        A = gen_hermite(eigv)
        t, A = runtime_error_lanczos(f,A,k)
        err = error(A[1][0], eigv[0])
        time_array.append(t)
        error_array.append(err)
    plt.plot(n_array, time_array, label = "runtime lanczos")
    #plt.plot(n_array, error_array, 'x', label = "errors")
    plt.legend()
    plt.show()
    
def selective_error(eig_1, eig_2, k = 1):
    n_1 = len(eig_1)
    n_2 = len(eig_2)
    error_1 = np.linalg.norm(eig_1[:k] - eig_2[:k])
    error_2 = np.linalg.norm(eig_1[:n_1-k:-1] - eig_2[:n_2-k:-1])
    return error_1 + error_2

# ------- testing starts here -------

n = 800

eigv = np.sort(5*np.random.rand(n)-0.5)
#eigv = np.array([-(i+1)*10000 for i in range(n)])
A = gen_hermite(eigv)

def matmul(v):
    return A@v


print(np.sort(eigv))
#print(np.round(QR_simple(A)[1],3))
#print(np.round(QR_shift(A)[1],3))
#print(np.round(QR_shift2(A)[1],3))
#print(np.round(QR_hesse(A)[0],3))
#print(np.round(lanczos(A,30)[1],3))
#print(np.linalg.eigvals(A))
#print(np.sort(np.round(arnoldi(matmul,n)[1], 3)))

#print(error(QR_simple(A)[1], eigv))
#print(error(QR_shift(A)[1], eigv))
#print(error(QR_shift2(A)[1], eigv))
#print(error(lanczos(A)[1], eigv))

#print(error(arnoldi(lambda x: A@x,n, 50)[1],eigv))
#print(arnoldi(lambda x: A@x,n, 50)[1])
#print(runtime(QR_simple, A))
#print(runtime(QR_shift, A))
#print(runtime(QR_shift2, A))
#print(runtime_error_lanczos(lanczos, A, 10)[0])


[-4.99719606e-01 -4.91053507e-01 -4.89723669e-01 -4.88130691e-01
 -4.87497232e-01 -4.78242884e-01 -4.61978070e-01 -4.50429159e-01
 -4.39731779e-01 -4.34847187e-01 -4.07191024e-01 -3.97874231e-01
 -3.97721554e-01 -3.94711894e-01 -3.92949749e-01 -3.90036546e-01
 -3.79434650e-01 -3.75420728e-01 -3.73303324e-01 -3.44840830e-01
 -3.41343491e-01 -3.20895454e-01 -3.10987436e-01 -3.08638443e-01
 -3.00496425e-01 -2.97353490e-01 -2.80474201e-01 -2.73594400e-01
 -2.59410399e-01 -2.54378247e-01 -2.44252248e-01 -2.39554393e-01
 -2.35405095e-01 -2.31765447e-01 -2.30338337e-01 -2.25998090e-01
 -2.25367856e-01 -2.12392230e-01 -2.01888279e-01 -1.95407208e-01
 -1.92941526e-01 -1.84235107e-01 -1.82134738e-01 -1.80009233e-01
 -1.75858629e-01 -1.70481490e-01 -1.65211974e-01 -1.63403102e-01
 -1.52714525e-01 -1.35702259e-01 -1.32034261e-01 -1.31558303e-01
 -1.27816049e-01 -1.20391575e-01 -9.58644753e-02 -9.07472925e-02
 -8.62166967e-02 -7.31544626e-02 -6.67113658e-02 -4.60672167e-02
 -4.59516964e-02 -2.87798

In [315]:
k = 120
for k in range(40,60):
    lanczos_test = np.array(lanczos(A,k)[1])
    #print(np.sort(lanczos_test))
    error = selective_error(lanczos_test, eigv, 1)


    print(error)

    arnoldi_test = arnoldi(lambda x: A@x,n,k)[1]
    #print(np.sort(arnoldi_test))
    error_1 = selective_error(arnoldi_test, eigv, 1)

    print(error_1)

C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:86: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:87: ComplexWarning: Casting complex values to real discards the imaginary part


0.003248233218010377


C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:124: ComplexWarning: Casting complex values to real discards the imaginary part
C:\Users\Salli\Anaconda3\lib\site-packages\ipykernel_launcher.py:127: ComplexWarning: Casting complex values to real discards the imaginary part


0.0067035421985401356
0.001750368446868933
0.003250722349330104
0.0028546606669752617
0.007120154458316819
0.006857368923881779
0.001967995475136597
0.0032573571689101244
0.00930767302941854
0.00037636859244355447
0.009224413517451069
0.0012207844678311375
0.00772877807351513
0.001716711513952529
0.0011549186915251752
0.0006704082933596256
0.0011007755002250263
0.00033346029605785255
0.0005194694160390312
0.0029892610832518707
0.0005127544587759658
0.0003538946851815439
0.0003983886559311145
0.00015424509948813725
0.00027228997689748846
0.0029114878329687954
0.00042338429098887076
0.00015666810102743511
0.0006712556697765493
0.00011874293572072814
0.00044187213507013157
0.0006140681519939406
0.00017719868379995063
0.009586909481443584
0.0003191872782371541
8.408695696449087e-05
4.177695115448188e-05
0.0004654325263029335
0.00017609383071004459


In [125]:
for k in range(20, 100):
    print(np.array(lanczos(A,k)[1]))
    print("----------------")

[ 4.48355744-4.06546761e-17j  4.40564401-3.94458778e-18j
  4.24933672-1.49738872e-17j  4.09005451-2.89753928e-17j
  3.86212856-1.70037833e-17j  3.62972269-2.31904921e-17j
  3.26797629-3.02903503e-17j  2.9292489 -3.54722305e-17j
  2.52148936-2.57030663e-17j  2.2453089 -3.26507529e-17j
  1.80078499-3.04119644e-17j  1.48950586-2.13641078e-17j
  1.140423  -3.26684163e-17j  0.72373219-3.52267002e-17j
  0.37245874-1.37813904e-17j  0.10376965-1.46366919e-17j
 -0.12285777-2.51493387e-17j -0.28103637-1.16841662e-17j
 -0.37480649+2.00130098e-19j -0.49512632-4.98754336e-17j]
----------------
[ 4.48509107+9.09931120e-18j  4.41662291+9.93830350e-18j
  4.27862345+1.08937226e-17j  4.12188881+1.21430795e-17j
  3.94470554-2.49250397e-18j  3.71564967+7.67735485e-18j
  3.36096615+2.61063381e-17j  3.05948208+7.59375130e-18j
  2.68819575-7.24070141e-18j  2.35432296+1.96073952e-17j
  2.04407879+2.76008586e-17j  1.66585806+1.63232744e-17j
  1.28879548-9.21389568e-18j  1.01443655+9.69012664e-18j
  0.69276388+

[ 4.48966483+2.10151682e-18j  4.46812979-1.20274381e-17j
  4.40917334-7.78786777e-18j  4.34519752+1.54318807e-17j
  4.24618043+1.62007975e-17j  4.1420802 +1.04368217e-17j
  4.01055351+3.24880495e-18j  3.84181682-1.60914666e-18j
  3.70390792-3.74131406e-18j  3.46510152+2.22985160e-17j
  3.2861603 +7.64618844e-18j  3.05717611+7.93807380e-18j
  2.84299707+1.46568034e-17j  2.56141096-1.62076277e-17j
  2.3453833 -1.65317617e-17j  2.16864399-1.59986516e-17j
  1.9127684 -7.01107090e-19j  1.65701874-6.41708871e-18j
  1.40706316-1.12270380e-17j  1.20807391+1.17257918e-17j
  0.99432309-5.89213877e-18j  0.7213629 +7.42715236e-19j
  0.52635159+1.59080111e-17j  0.32970541+2.28426790e-18j
  0.1611419 +8.84072393e-18j  0.04731119+1.63906167e-17j
 -0.08699074+1.00750253e-17j -0.21082378+1.10864201e-17j
 -0.32189243-1.32978085e-17j -0.37251974-6.52683264e-18j
 -0.47218515-1.86200146e-17j -0.49826011-1.00753988e-17j]
----------------
[ 4.48804354+3.48042912e-17j  4.4656714 +5.88792806e-18j
  4.41802334+

[ 4.49405864+6.59604700e-18j  4.48338108+4.02270129e-18j
  4.44082873+1.62958093e-17j  4.39699752-2.49312089e-18j
  4.30183296-2.79089899e-17j  4.24968317-2.09828780e-17j
  4.16787413+3.38816023e-18j  4.05440265-8.79937183e-18j
  3.96291174-8.77949863e-19j  3.80889907-3.90005106e-18j
  3.70313124+9.38917144e-18j  3.53831489-2.05464806e-18j
  3.38452516-1.25001154e-17j  3.22020465+6.17702952e-18j
  3.04235996+6.49047858e-18j  2.84594188+1.16372442e-18j
  2.63189684-1.11199220e-17j  2.48134525-3.47623739e-19j
  2.3010303 +5.61708011e-18j  2.13291389-1.86705901e-18j
  1.93229535+4.90498556e-18j  1.71318477-1.49732547e-18j
  1.57219146+2.48286107e-18j  1.36142567-8.64765853e-18j
  1.19865863-2.42108685e-18j  1.0034009 +1.33232914e-17j
  0.76284432+2.52451474e-18j  0.67024793-1.17421221e-17j
  0.46575373+8.21470684e-18j  0.32652134+9.02754464e-18j
  0.17705819-1.07868078e-17j  0.08993307+9.13408958e-18j
 -0.04261434-1.38297306e-17j -0.14421678-3.40284533e-17j
 -0.22129237-6.75671874e-18j -0

[ 4.49591039+7.93878175e-18j  4.48425597+7.36747140e-18j
  4.44219325-5.84424576e-18j  4.41362008-3.06571839e-18j
  4.37766628+2.78541671e-19j  4.31831056-9.92021419e-18j
  4.25828941-3.21657668e-18j  4.18018251+4.19825017e-19j
  4.08396596-5.94567394e-18j  3.99989456+2.82813807e-18j
  3.89285437+5.66639956e-18j  3.77719888-1.06054670e-17j
  3.68058212+3.24407725e-18j  3.55323342+4.72932009e-18j
  3.42019016+1.38722721e-17j  3.28094661+1.48946478e-17j
  3.10928936+2.28549845e-18j  2.99176859+3.67559293e-19j
  2.83094681-1.29054544e-17j  2.64029404-9.68215239e-19j
  2.50434116-1.06912348e-18j  2.34680215+3.97114154e-19j
  2.21960291+8.57240700e-18j  2.04001167+2.25784414e-17j
  1.88616635+2.35009124e-18j  1.68270248-4.55506857e-18j
  1.5713513 -8.67398124e-19j  1.3746501 -5.85399772e-18j
  1.22487498-1.52929533e-17j  1.09226351+1.79104005e-18j
  0.9277961 -2.62955938e-19j  0.74627782+1.16400563e-17j
  0.64715072+4.02638790e-18j  0.48199236-2.75559931e-18j
  0.380197  -1.91630956e-17j  0

[ 4.49228618+7.89641723e-18j  4.48388671-5.02112032e-18j
  4.46259782+3.13760140e-17j  4.44018928+1.32220079e-17j
  4.39778209+5.75326205e-18j  4.34533959+1.53860769e-17j
  4.29824424+2.23770415e-17j  4.25028533+2.14214904e-17j
  4.17882867+1.78626886e-17j  4.09740548+3.02023856e-17j
  4.01828145+2.56561352e-17j  3.95483748+1.36707379e-17j
  3.83721723+2.87335264e-17j  3.73362243+1.60165633e-17j
  3.64068327+2.03226202e-17j  3.48418733+1.75657840e-17j
  3.40339957+1.17356553e-17j  3.26986763+1.28319476e-17j
  3.13985147+1.58384160e-17j  3.01217248+1.57242165e-17j
  2.86042394+1.13391347e-17j  2.71520098+1.06941699e-17j
  2.56588597+1.08374327e-17j  2.43609211+1.35945267e-17j
  2.30167359+1.90714999e-17j  2.16853848+2.00350145e-17j
  2.02796835+2.54463398e-17j  1.88151368+1.40531549e-17j
  1.73406182+2.13322597e-17j  1.59542585+1.29883021e-17j
  1.44778364+1.71047838e-17j  1.31792401+1.12902477e-17j
  1.19459755+3.54544971e-18j  1.06690522+2.40559525e-17j
  0.90140218+2.05470485e-17j  0

...

[ 4.49638631-1.99229126e-17j  4.48947207-1.46793745e-17j
  4.48276294-1.71873343e-17j  4.44634349+4.66235892e-18j
  4.43608474+9.22946128e-19j  4.4076109 -6.45914776e-18j
  4.39753242+1.73872718e-18j  4.36551513-5.71717510e-18j
  4.32407136-8.65044834e-18j  4.29404133+1.41630648e-17j
  4.26498277-2.61363931e-18j  4.23251075+7.23544762e-18j
  4.18009924-9.79566510e-20j  4.15435461+5.46620578e-18j
  4.09521511-8.31234247e-18j  4.05215562-1.62160256e-17j
  3.99419758+1.15118927e-18j  3.95860775-6.13649026e-18j
  3.88119213+2.49937207e-18j  3.83812499-1.30671316e-17j
  3.73762709-1.26148902e-18j  3.6886427 -1.05373162e-17j
  3.59437164-4.06897127e-18j  3.54524933-7.81460513e-18j
  3.46361412-3.32574184e-19j  3.40436462-1.24243885e-17j
  3.29050418-3.29599624e-18j  3.23724217+1.08537177e-17j
  3.14671516+2.90567890e-18j  3.04722136-1.04399830e-17j
  2.96558963-1.65365568e-18j  2.88440666+6.82602458e-18j
  2.82926904+3.81086538e-18j  2.72264043+3.01406300e-18j
  2.60339194-2.84596325e-18j  2

[ 4.49638632-4.56288455e-18j  4.48947208+2.88267881e-18j
  4.48276296-4.42986851e-18j  4.44637544+6.55802623e-18j
  4.43580934+6.49675507e-19j  4.40932834-1.73672351e-17j
  4.40181063-2.95228705e-18j  4.38740251-8.32844761e-18j
  4.36429877-1.47169363e-17j  4.32305918-8.69820196e-18j
  4.2904223 -1.83327400e-17j  4.26438078-4.03107287e-19j
  4.20323895-9.92390644e-18j  4.1717688 +1.87959923e-18j
  4.13195449-7.60017612e-18j  4.07450449-1.00274796e-17j
  4.04767676+1.45147407e-18j  3.99478317+7.22527724e-20j
  3.95785209-3.66909604e-18j  3.87445515-1.40225195e-17j
  3.83811373+6.93542190e-18j  3.74052301+7.00996708e-18j
  3.69777606-4.25975236e-18j  3.63484661-1.16010799e-17j
  3.57052342-1.38445424e-17j  3.46216476-1.90075692e-17j
  3.40359984-7.86876737e-18j  3.3250651 -2.51459752e-18j
  3.27266586-1.34982160e-17j  3.17570235-9.94592334e-18j
  3.10388291-1.87104189e-17j  3.04066698-9.10594843e-18j
  2.93387239-8.51419094e-18j  2.85001274-7.41909568e-18j
  2.74777045-1.02440257e-17j  2

[ 4.49638632+5.25555796e-18j  4.48947209+7.10685726e-18j
  4.48276297+1.64777323e-17j  4.44638834+1.13577975e-17j
  4.43597253+1.52081015e-17j  4.43307465+5.70255297e-17j
  4.40852306+5.49598720e-18j  4.39605724+9.67497398e-18j
  4.36536287+2.84817189e-18j  4.32367452+2.71356115e-18j
  4.29592828+7.90808907e-18j  4.26633946+4.48509846e-18j
  4.20056948+4.32805936e-18j  4.18024231+1.49778188e-17j
  4.15758516+1.69487343e-17j  4.10329741+1.40581948e-17j
  4.06152894+6.67589466e-18j  4.02253429+7.73624952e-18j
  3.97038191+6.99232926e-18j  3.9047341 +1.47208889e-17j
  3.84599715+2.79992916e-18j  3.81255944+1.69638086e-17j
  3.74095095+7.20200207e-18j  3.70023062+9.20297784e-18j
  3.61470602+2.29330359e-17j  3.5711655 +3.79205667e-18j
  3.47252141+8.46010215e-19j  3.4132388 +2.70376787e-18j
  3.33744227+2.02583599e-17j  3.27863149+6.59623900e-18j
  3.19758345+2.14209845e-18j  3.13569883+9.13311920e-18j
  3.04079531+1.08196332e-17j  2.94762614+1.21845816e-17j
  2.87642841+2.16309743e-17j  2

[ 4.49638632+1.28421008e-17j  4.48947209+5.84614890e-18j
  4.48276297+5.98872664e-18j  4.44639515-1.50373971e-18j
  4.43692383-1.75169410e-17j  4.43430424-7.31897821e-17j
  4.4082714 -7.18482587e-18j  4.3961921 -2.91654697e-17j
  4.36541089+9.91582103e-18j  4.32394291-1.34653884e-17j
  4.29643177-1.68883527e-17j  4.26714826-6.06969235e-18j
  4.24173897-1.99901188e-17j  4.19491546-8.98557494e-18j
  4.1714484 -6.90343783e-18j  4.12413436-3.77093724e-18j
  4.09414241-2.48516265e-17j  4.05479306+5.66801175e-18j
  4.00510835+8.80927765e-20j  3.96599941-5.41986255e-19j
  3.90232519-1.34393314e-17j  3.84388515+6.78004657e-18j
  3.80908041-1.89435929e-17j  3.73940063-3.78210844e-18j
  3.69192875-7.75428424e-18j  3.6423397 -3.19165019e-17j
  3.57421088+2.14065185e-18j  3.48470259-1.27039936e-17j
  3.41493267-3.81148468e-18j  3.33902236-2.05607955e-17j
  3.27815917-2.60083533e-18j  3.20649374-1.03091976e-17j
  3.14773017-6.96273976e-19j  3.07126427-2.55856190e-17j
  3.03040523-1.31053247e-17j  2

[ 4.49638632-6.90287602e-18j  4.48947209-4.15135726e-17j
  4.48276297-5.10197573e-19j  4.44639557-1.40531986e-17j
  4.43710786-7.84666872e-20j  4.43492729-3.03052836e-18j
  4.40680554-9.59640976e-19j  4.39664421-1.74026512e-17j
  4.36541311-1.98905525e-17j  4.32421489-6.82495619e-18j
  4.30758336-3.70612664e-18j  4.29458878-1.04162747e-17j
  4.26698419+8.49519381e-18j  4.22512257+4.03851608e-18j
  4.19507797-4.99486349e-18j  4.16728717-5.00209531e-18j
  4.12504953-1.00259714e-17j  4.09216668-1.55005777e-18j
  4.0575777 -6.66082631e-18j  4.0014196 +7.02380096e-18j
  3.96217942-1.18045987e-17j  3.89130074+2.91864150e-18j
  3.84106807-6.57894989e-18j  3.79849601-1.29540687e-17j
  3.7391626 +7.57632314e-18j  3.69767575-1.07815478e-17j
  3.64504636-8.91142239e-18j  3.57272287-9.48624980e-18j
  3.48083017-2.85968905e-18j  3.42179465+4.43781197e-19j
  3.3811775 -2.68146019e-18j  3.28852117-4.33874416e-19j
  3.2501537 +5.10538126e-18j  3.18186221-6.65566064e-18j
  3.119132  -2.09246465e-18j  3

In [47]:
%matplotlib notebook

testing(QR_shift2, 10, 100)

testing_lanczos(lanczos, 7, 10, 100)

TypeError: 'numpy.float64' object is not callable

1.6340957333511992e-14
2.847873245036576e-14
4.6120200349769874e-14
3.4225736066282965e-14
4.9544368613970536e-14
2.9927799515058673e-14
3.2250645877544357e-14
5.0872835944216865e-14
5.917642274415065e-14
5.654984554072407e-14
5.370129224611537e-14
7.696323896657158e-14
8.11477294175874e-14
9.146223388691494e-14
9.721311472393263e-14
9.380573530943866e-14
9.945190618861547e-14
1.0843844264475484e-13
1.0320707350582293e-13
1.1061865972903026e-13
1.0691396438941555e-13
1.1980732211582484e-13
1.074059993037486e-13
1.4276638666545316e-13
1.994325271203095e-13
1.8686931595554436e-13
1.5813295095384266e-13
1.625972658974202e-13
1.643010819750149e-13
1.7868635202078483e-13
2.1589452692194758e-13
1.788671326745715e-13
1.9458838519943438e-13
2.1183766342160963e-13
2.2198917496212178e-13
2.203421624066143e-13
2.617295920337076e-13
2.4308383025512584e-13
2.6382737167320084e-13
2.6807442211089184e-13
2.454120962250446e-13
2.9032033110685434e-13
3.1178814909549736e-13
2.6591266677548974e-13
2.89110